In [1]:
!apt-get -y install mysql-server

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcgi-fast-perl libcgi-pm-perl libclone-perl libencode-locale-perl libfcgi-bin libfcgi-perl
  libfcgi0ldbl libhtml-parser-perl libhtml-tagset-perl libhtml-template-perl libhttp-date-perl
  libhttp-message-perl libio-html-perl liblwp-mediatypes-perl libmecab2 libprotobuf-lite23
  liburi-perl mecab-ipadic mecab-ipadic-utf8 mecab-utils mysql-client-8.0 mysql-client-core-8.0
  mysql-server-8.0 mysql-server-core-8.0
Suggested packages:
  libdata-dump-perl libipc-sharedcache-perl libbusiness-isbn-perl libwww-perl mailx tinyca
The following NEW packages will be installed:
  libcgi-fast-perl libcgi-pm-perl libclone-perl libencode-locale-perl libfcgi-bin libfcgi-perl
  libfcgi0ldbl libhtml-parser-perl libhtml-tagset-perl libhtml-template-perl libhttp-date-perl
  libhttp-message-perl libio-html-perl liblwp-mediatypes-perl libmecab2 libprotobuf-l

In [2]:
!service mysql start

 * Starting MySQL database server mysqld
su: warning: cannot change directory to /nonexistent: No such file or directory
   ...done.


In [3]:
!mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH 'mysql_native_password' BY 'root';FLUSH PRIVILEGES;"

In [4]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 42.6 MB/s eta 0:00:00


In [5]:
#1.
import mysql.connector

# Establish a connection to the MySQL server
conn = mysql.connector.connect(
    host='localhost',        # Replace with your host
    user='root',    # Replace with your MySQL username
    password='root' # Replace with your MySQL password
)

cursor = conn.cursor()

# Create the database
cursor.execute("CREATE DATABASE IF NOT EXISTS college;")

# Use the college database
cursor.execute("USE college;")

# Create the student table
cursor.execute("""
CREATE TABLE IF NOT EXISTS student (
    roll INT PRIMARY KEY,
    name VARCHAR(50),
    class VARCHAR(10),
    marks DECIMAL(5, 2),
    age INT
);
""")

# Insert rows into the student table
cursor.execute("""
INSERT INTO student (roll, name, class, marks, age) VALUES
(1, 'Alice', '10th', 85.5, 15),
(2, 'Bob', '10th', 78.0, 16),
(3, 'Charlie', '11th', 92.0, 16),
(4, 'David', '12th', 88.5, 17),
(5, 'Eva', '12th', 91.0, 17);
""")

# Commit the changes
conn.commit()

# Delete the entry with roll number 4
cursor.execute("DELETE FROM student WHERE roll = 4;")

# Update the marks of roll number 3
cursor.execute("UPDATE student SET marks = 95.0 WHERE roll = 3;")

# Give additional grace marks of 0.5 to all students
cursor.execute("UPDATE student SET marks = marks + 0.5;")

# Truncate the table
cursor.execute("TRUNCATE TABLE student;")

# Insert the data again using multi-insert statement
cursor.execute("""
INSERT INTO student (roll, name, class, marks, age) VALUES
(1, 'Alice', '10th', 85.5, 15),
(2, 'Bob', '10th', 78.0, 16),
(3, 'Charlie', '11th', 92.0, 16),
(4, 'David', '12th', 88.5, 17),
(5, 'Eva', '12th', 91.0, 17);
""")

# Commit the changes
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

print("Database operations completed successfully.")

Database operations completed successfully.


In [6]:
#2.
import mysql.connector

# Establish a connection to the MySQL server
conn = mysql.connector.connect(
    host='localhost',        # Replace with your host
    user='root',    # Replace with your MySQL username
    password='root' # Replace with your MySQL password
)

cursor = conn.cursor()

# Create the database
cursor.execute("CREATE DATABASE IF NOT EXISTS college;")

# Use the college database
cursor.execute("USE college;")

# Alter table to add and drop constraints
cursor.execute("ALTER TABLE student ADD CONSTRAINT unique_name UNIQUE (name);")
cursor.execute("ALTER TABLE student ADD CONSTRAINT check_marks CHECK (marks BETWEEN 0 AND 120);")
cursor.execute("ALTER TABLE student DROP CONSTRAINT unique_name;")

# Aggregate functions
cursor.execute("SELECT SUM(marks) AS total_marks FROM student;")
total_marks = cursor.fetchone()[0]
print(f"Total Marks: {total_marks}")

cursor.execute("SELECT AVG(marks) AS average_marks FROM student;")
average_marks = cursor.fetchone()[0]
print(f"Average Marks: {average_marks}")

cursor.execute("SELECT COUNT(*) AS total_students FROM student;")
total_students = cursor.fetchone()[0]
print(f"Total Students: {total_students}")

cursor.execute("SELECT MAX(marks) AS highest_marks FROM student;")
highest_marks = cursor.fetchone()[0]
print(f"Highest Marks: {highest_marks}")

cursor.execute("SELECT MIN(marks) AS lowest_marks FROM student;")
lowest_marks = cursor.fetchone()[0]
print(f"Lowest Marks: {lowest_marks}")

# Group by and having
cursor.execute("""
SELECT class, AVG(marks) AS average_marks
FROM student
GROUP BY class;
""")
group_by_class = cursor.fetchall()
print("Average Marks by Class:")
for row in group_by_class:
    print(f"Class: {row[0]}, Average Marks: {row[1]}")

cursor.execute("""
SELECT class, AVG(marks) AS average_marks
FROM student
GROUP BY class
HAVING AVG(marks) > 80;
""")
having_clause = cursor.fetchall()
print("Classes with Average Marks > 80:")
for row in having_clause:
    print(f"Class: {row[0]}, Average Marks: {row[1]}")

# LIKE, DISTINCT, ORDER BY, BETWEEN...AND
cursor.execute("SELECT * FROM student WHERE name LIKE 'A%';")
like_clause = cursor.fetchall()
print("Students with names starting with 'A':")
for row in like_clause:
    print(row)

cursor.execute("SELECT DISTINCT class FROM student;")
distinct_classes = cursor.fetchall()
print("Distinct Classes:")
for row in distinct_classes:
    print(row[0])

cursor.execute("SELECT * FROM student ORDER BY marks DESC;")
order_by_marks = cursor.fetchall()
print("Students Ordered by Marks (Descending):")
for row in order_by_marks:
    print(row)

cursor.execute("SELECT * FROM student WHERE marks BETWEEN 80 AND 90;")
between_and_clause = cursor.fetchall()
print("Students with Marks Between 80 and 90:")
for row in between_and_clause:
    print(row)

# NULL comparison, IN/NOT IN
cursor.execute("SELECT * FROM student WHERE marks IS NULL;")
null_comparison = cursor.fetchall()
print("Students with NULL Marks:")
for row in null_comparison:
    print(row)

cursor.execute("SELECT * FROM student WHERE roll IN (1, 2, 3);")
in_clause = cursor.fetchall()
print("Students with Roll Numbers IN (1, 2, 3):")
for row in in_clause:
    print(row)

cursor.execute("SELECT * FROM student WHERE roll NOT IN (1, 2, 3);")
not_in_clause = cursor.fetchall()
print("Students with Roll Numbers NOT IN (1, 2, 3):")
for row in not_in_clause:
    print(row)

# Close the cursor and connection
cursor.close()
conn.close()

print("Database operations completed successfully.")


Total Marks: 435.00
Average Marks: 87.000000
Total Students: 5
Highest Marks: 92.00
Lowest Marks: 78.00
Average Marks by Class:
Class: 10th, Average Marks: 81.750000
Class: 11th, Average Marks: 92.000000
Class: 12th, Average Marks: 89.750000
Classes with Average Marks > 80:
Class: 10th, Average Marks: 81.750000
Class: 11th, Average Marks: 92.000000
Class: 12th, Average Marks: 89.750000
Students with names starting with 'A':
(1, 'Alice', '10th', Decimal('85.50'), 15)
Distinct Classes:
10th
11th
12th
Students Ordered by Marks (Descending):
(3, 'Charlie', '11th', Decimal('92.00'), 16)
(5, 'Eva', '12th', Decimal('91.00'), 17)
(4, 'David', '12th', Decimal('88.50'), 17)
(1, 'Alice', '10th', Decimal('85.50'), 15)
(2, 'Bob', '10th', Decimal('78.00'), 16)
Students with Marks Between 80 and 90:
(1, 'Alice', '10th', Decimal('85.50'), 15)
(4, 'David', '12th', Decimal('88.50'), 17)
Students with NULL Marks:
Students with Roll Numbers IN (1, 2, 3):
(1, 'Alice', '10th', Decimal('85.50'), 15)
(2, 'Bob

In [27]:
import mysql.connector

# Establish a connection to the MySQL server
conn = mysql.connector.connect(
    host='localhost',        # Replace with your host
    user='root',    # Replace with your MySQL username
    password='root' # Replace with your MySQL password
)

cursor = conn.cursor()

# Create the database
cursor.execute("CREATE DATABASE IF NOT EXISTS college;")

# Use the college database
cursor.execute("USE college;")

cursor = conn.cursor()

# Create the employee table
cursor.execute("""
CREATE TABLE IF NOT EXISTS employees (
    Employee_id INT PRIMARY KEY,
    First_name VARCHAR(50),
    Last_name VARCHAR(50),
    Salary INT,
    Joining_date DATE,
    Departement VARCHAR(50)
);
""")

# Insert data into the employee table
cursor.execute("""
INSERT INTO employees (Employee_id, First_name, Last_name, Salary, Joining_date, Departement) VALUES
(1, 'Bob', 'Kinto', 1000000, '2019-01-20', 'Finance'),
(2, 'Jerry', 'Kansxo', 6000000, '2019-01-15', 'IT'),
(3, 'Philip', 'Jose', 8900000, '2019-02-05', 'Banking'),
(4, 'John', 'Abraham', 2000000, '2019-02-25', 'Insurance'),
(5, 'Michael', 'Mathew', 2200000, '2019-02-28', 'Finance'),
(6, 'Alex', 'chreketo', 4000000, '2019-05-10', 'IT'),
(7, 'Yohan', 'Soso', 1230000, '2019-06-20', 'Banking');
""")

# Commit the changes to the database
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

print("Table created and rows inserted successfully.")

IntegrityError: 1062 (23000): Duplicate entry '1' for key 'employees.PRIMARY'

In [24]:
#3
import mysql.connector

# Establish a connection to the MySQL database
connection = mysql.connector.connect(
    host='localhost',        # Replace with your host
    user='root',    # Replace with your MySQL username
    password='root' # Replace with your MySQL password
)

cursor = connection.cursor(dictionary=True)  # To get results as dictionaries

# Use the college database
cursor.execute("USE college;")


#Basic queries
cursor.execute("SELECT First_name, Last_name FROM employees;")
result = cursor.fetchall()
for row in result:
    print(row)

cursor.execute("SELECT COUNT(*) AS total_employees FROM employees WHERE Departement = 'IT';")
result = cursor.fetchone()
print(result)
cursor.execute("SELECT COUNT(*) AS total_employees FROM employees WHERE Departement = 'IT';")
result = cursor.fetchone()
print(result)

cursor.execute("SELECT * FROM employees WHERE Joining_date > '2019-02-01';")
result = cursor.fetchall()
for row in result:
    print(row)

#Aggregation and Grouping
cursor.execute("SELECT AVG(Salary) AS average_salary FROM employees WHERE Departement = 'Finance';")
result = cursor.fetchone()
print(result)

cursor.execute("SELECT MAX(Salary) AS max_salary FROM employees WHERE Departement = 'Banking';")
result = cursor.fetchone()
print(result)

cursor.execute("SELECT Departement, SUM(Salary) AS total_salary FROM employees GROUP BY Departement;")
result = cursor.fetchall()
for row in result:
    print(row)

#Filtering and Sorting
cursor.execute("SELECT * FROM employees WHERE Salary = (SELECT MAX(Salary) FROM employees);")
result = cursor.fetchall()
for row in result:
    print(row)

cursor.execute("SELECT * FROM employees WHERE Salary > 3000000 ORDER BY Joining_date;")
result = cursor.fetchall()
for row in result:
    print(row)

cursor.execute("SELECT * FROM employees WHERE Last_name LIKE 'K%';")
result = cursor.fetchall()
for row in result:
    print(row)

#Subqueries and Joins
cursor.execute("SELECT MAX(Salary) AS second_highest_salary FROM employees WHERE Salary < (SELECT MAX(Salary) FROM employees);")
result = cursor.fetchone()
print(result)

cursor.execute("SELECT * FROM employees WHERE Salary > (SELECT AVG(Salary) FROM employees);")
result = cursor.fetchall()
for row in result:
    print(row)

cursor.execute("UPDATE employees SET Salary = Salary * 1.10 WHERE Departement = 'Insurance';")
connection.commit()
print("Salaries updated.")

#Updating and Deleting Records




{'First_name': 'Bob', 'Last_name': 'Kinto'}
{'First_name': 'Jerry', 'Last_name': 'Kansxo'}
{'First_name': 'Philip', 'Last_name': 'Jose'}
{'First_name': 'John', 'Last_name': 'Abraham'}
{'First_name': 'Michael', 'Last_name': 'Mathew'}
{'First_name': 'Alex', 'Last_name': 'chreketo'}
{'First_name': 'Yohan', 'Last_name': 'Soso'}
{'total_employees': 2}
{'total_employees': 2}
{'Employee_id': 3, 'First_name': 'Philip', 'Last_name': 'Jose', 'Salary': 8900000, 'Joining_date': datetime.date(2019, 2, 5), 'Departement': 'Banking'}
{'Employee_id': 4, 'First_name': 'John', 'Last_name': 'Abraham', 'Salary': 2000000, 'Joining_date': datetime.date(2019, 2, 25), 'Departement': 'Insurance'}
{'Employee_id': 5, 'First_name': 'Michael', 'Last_name': 'Mathew', 'Salary': 2200000, 'Joining_date': datetime.date(2019, 2, 28), 'Departement': 'Finance'}
{'Employee_id': 6, 'First_name': 'Alex', 'Last_name': 'chreketo', 'Salary': 4000000, 'Joining_date': datetime.date(2019, 5, 10), 'Departement': 'IT'}
{'Employee_id'

In [26]:
# Establish a connection to the MySQL database
conn = mysql.connector.connect(
    host='localhost',        # Replace with your host
    user='root',    # Replace with your MySQL username
    password='root', # Replace with your MySQL password
    database='college'  # Replace with your database name
)

cursor = conn.cursor()

# Q16: Update the salary of employees in the Insurance department by a 10% increase
cursor.execute("""
UPDATE employee
SET Salary = Salary * 1.10
WHERE Departement = 'Insurance';
""")
print("Salaries of employees in the Insurance department updated.")

# Q17: Delete records of employees who joined before 2019-02-01
cursor.execute("""
DELETE FROM employee
WHERE Joining_date < '2019-02-01';
""")
print("Records of employees who joined before 2019-02-01 deleted.")

# Commit the changes to the database
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

print("Update and deletion operations completed successfully.")

Salaries of employees in the Insurance department updated.
Records of employees who joined before 2019-02-01 deleted.
Update and deletion operations completed successfully.
